## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Wladyslawowo,PL,54.79,18.40,64.99,100,65,3.36,mist
1,1,Northam,GB,51.03,-4.22,61.00,90,53,11.01,broken clouds
2,2,Chuy,UY,-33.70,-53.46,46.26,71,0,6.69,clear sky
3,3,Lodwar,KE,3.12,35.60,79.66,49,82,24.36,broken clouds
4,4,Dwarka,IN,22.24,68.97,83.28,86,100,19.19,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Lodwar,KE,3.12,35.60,79.66,49,82,24.36,broken clouds
4,4,Dwarka,IN,22.24,68.97,83.28,86,100,19.19,light rain
8,8,Pemangkat,ID,1.17,108.97,77.23,88,100,10.47,light rain
25,25,Albany,US,42.60,-73.97,82.99,61,0,6.17,clear sky
26,26,Bethel,US,41.37,-73.41,82.40,58,1,3.36,clear sky
27,27,Waingapu,ID,-9.66,120.26,79.95,61,26,9.33,scattered clouds
35,35,Vaini,TO,-21.20,-175.20,78.80,88,75,9.17,broken clouds
38,38,Tucuma,BR,-0.18,-50.05,78.10,86,2,6.31,clear sky
40,40,Cockburn Town,TC,21.46,-71.14,84.02,79,82,27.96,broken clouds
41,41,Kailua,US,21.40,-157.74,84.20,66,40,11.41,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Lodwar,KE,3.12,35.60,79.66,49,82,24.36,broken clouds
4,4,Dwarka,IN,22.24,68.97,83.28,86,100,19.19,light rain
8,8,Pemangkat,ID,1.17,108.97,77.23,88,100,10.47,light rain
25,25,Albany,US,42.60,-73.97,82.99,61,0,6.17,clear sky
26,26,Bethel,US,41.37,-73.41,82.40,58,1,3.36,clear sky
27,27,Waingapu,ID,-9.66,120.26,79.95,61,26,9.33,scattered clouds
35,35,Vaini,TO,-21.20,-175.20,78.80,88,75,9.17,broken clouds
38,38,Tucuma,BR,-0.18,-50.05,78.10,86,2,6.31,clear sky
40,40,Cockburn Town,TC,21.46,-71.14,84.02,79,82,27.96,broken clouds
41,41,Kailua,US,21.40,-157.74,84.20,66,40,11.41,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Lodwar,KE,79.66,broken clouds,3.12,35.60,
4,Dwarka,IN,83.28,light rain,22.24,68.97,
8,Pemangkat,ID,77.23,light rain,1.17,108.97,
25,Albany,US,82.99,clear sky,42.60,-73.97,
26,Bethel,US,82.40,clear sky,41.37,-73.41,
27,Waingapu,ID,79.95,scattered clouds,-9.66,120.26,
35,Vaini,TO,78.80,broken clouds,-21.20,-175.20,
38,Tucuma,BR,78.10,clear sky,-0.18,-50.05,
40,Cockburn Town,TC,84.02,broken clouds,21.46,-71.14,
41,Kailua,US,84.20,scattered clouds,21.40,-157.74,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}" # possibly unnecessary step?
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Lodwar,KE,79.66,broken clouds,3.12,35.60,Turkwel Lodge
4,Dwarka,IN,83.28,light rain,22.24,68.97,Hotel Narayan Avezika Comfort
8,Pemangkat,ID,77.23,light rain,1.17,108.97,Kost H.Syamsuddin
25,Albany,US,82.99,clear sky,42.60,-73.97,
26,Bethel,US,82.40,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
27,Waingapu,ID,79.95,scattered clouds,-9.66,120.26,Pondok Wisata Elim
35,Vaini,TO,78.80,broken clouds,-21.20,-175.20,Keleti Beach Resort
38,Tucuma,BR,78.10,clear sky,-0.18,-50.05,
40,Cockburn Town,TC,84.02,broken clouds,21.46,-71.14,Osprey Beach Hotel
41,Kailua,US,84.20,scattered clouds,21.40,-157.74,Sheffield House Bed & Breakfast


In [10]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# Import numpy as np
import numpy as np

In [12]:
# Replace empty Hotel Names with NaN for replacement
hotel_df.loc[(hotel_df["Hotel Name"] == "")] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Lodwar,KE,79.66,broken clouds,3.12,35.60,Turkwel Lodge
4,Dwarka,IN,83.28,light rain,22.24,68.97,Hotel Narayan Avezika Comfort
8,Pemangkat,ID,77.23,light rain,1.17,108.97,Kost H.Syamsuddin
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Bethel,US,82.40,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
27,Waingapu,ID,79.95,scattered clouds,-9.66,120.26,Pondok Wisata Elim
35,Vaini,TO,78.80,broken clouds,-21.20,-175.20,Keleti Beach Resort
38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Cockburn Town,TC,84.02,broken clouds,21.46,-71.14,Osprey Beach Hotel
41,Kailua,US,84.20,scattered clouds,21.40,-157.74,Sheffield House Bed & Breakfast


In [13]:
hotel_df.isnull().sum()

City                   17
Country                17
Max Temp               17
Current Description    17
Lat                    17
Lng                    17
Hotel Name             17
dtype: int64

In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Lodwar,KE,79.66,broken clouds,3.12,35.60,Turkwel Lodge
4,Dwarka,IN,83.28,light rain,22.24,68.97,Hotel Narayan Avezika Comfort
8,Pemangkat,ID,77.23,light rain,1.17,108.97,Kost H.Syamsuddin
26,Bethel,US,82.40,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
27,Waingapu,ID,79.95,scattered clouds,-9.66,120.26,Pondok Wisata Elim
35,Vaini,TO,78.80,broken clouds,-21.20,-175.20,Keleti Beach Resort
40,Cockburn Town,TC,84.02,broken clouds,21.46,-71.14,Osprey Beach Hotel
41,Kailua,US,84.20,scattered clouds,21.40,-157.74,Sheffield House Bed & Breakfast
44,Butaritari,KI,83.21,overcast clouds,3.07,172.79,Isles Sunset Lodge
45,Kulhudhuffushi,MV,82.85,scattered clouds,6.62,73.07,Haajy Guest House 1


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))